In [ ]:
"""
Author: Matt Martin
Date: 1/27/24
Desc: Various techniques to generate test data in spark
"""

## create the spark connection/instance
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("sequence") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.instances", 10) \
    .getOrCreate()

In [4]:
#generate row sequence
df = spark.range(1,1_000_001).toDF('row_id')
#generate a list of dates
start_dt = "2001-01-01"
end_dt = "2024-05-10"
dates = spark.sql(f"SELECT explode(sequence(to_date('{start_dt}'), to_date('{end_dt}'), interval 1 day)) as date")

In [5]:
df.createOrReplaceTempView('numbers')
dates.createOrReplaceTempView('dates')

In [6]:
sql = """
select n.row_id, d.date
from numbers as n, dates as d
limit 10000
"""
spark.sql(sql).show(10)

+------+----------+
|row_id|      date|
+------+----------+
|     1|2001-01-01|
|     1|2001-01-02|
|     1|2001-01-03|
|     1|2001-01-04|
|     1|2001-01-05|
|     1|2001-01-06|
|     1|2001-01-07|
|     1|2001-01-08|
|     1|2001-01-09|
|     1|2001-01-10|
+------+----------+
only showing top 10 rows



In [ ]:
spark.stop